In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

train_data_dir = '/content/drive/MyDrive/trainnew'
val_data_dir = '/content/drive/MyDrive/valnew'

# Parameters
num_classes = 2
img_width, img_height = 224, 224
batch_size = 32
epochs = 30

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples ,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the trained model
model.save('poacher_detection_model.h5')

Found 491 images belonging to 2 classes.
Found 116 images belonging to 2 classes.
9406464/9406464 [==============================] - 0s 0us/step


Epoch 1/30
15/15 [==============================] - 129s 8s/step - loss: 0.7341 - accuracy: 0.7603 - val_loss: 0.1875 - val_accuracy: 0.9271
Epoch 2/30
15/15 [==============================] - 32s 2s/step - loss: 0.1878 - accuracy: 0.9259 - val_loss: 0.2046 - val_accuracy: 0.8958
Epoch 3/30
15/15 [==============================] - 33s 2s/step - loss: 0.1582 - accuracy: 0.9354 - val_loss: 0.2372 - val_accuracy: 0.9062
Epoch 4/30
15/15 [==============================] - 36s 2s/step - loss: 0.1189 - accuracy: 0.9521 - val_loss: 0.2334 - val_accuracy: 0.9167
Epoch 5/30
15/15 [==============================] - 34s 2s/step - loss: 0.0939 - accuracy: 0.9586 - val_loss: 0.1348 - val_accuracy: 0.9271
Epoch 6/30
15/15 [==============================] - 30s 2s/step - loss: 0.0709 - accuracy: 0.9760 - val_loss: 0.1334 - val_accuracy: 0.9375
Epoch 7/30
15/15 [==============================] - 31s 2s/step - loss: 0.0515 - accuracy: 0.9826 - val_loss: 0.1774 - val_accuracy: 0.9271
Epoch 8/30
15/15 [=

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('poacher_detection_model.h5')


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    return img


In [ ]:
new_image_paths = [
    '/content/drive/MyDrive/val/val1/img231.jpg',
    '/content/drive/MyDrive/val/val1/img228.jpg',
    '/content/drive/MyDrive/val/val1/i2.jpg',
    '/content/drive/MyDrive/val/val1/img208.jpg',
    '/content/drive/MyDrive/val/val1/img202.jpg',
    '/content/drive/MyDrive/train/train1/056.jpg'

]



In [ ]:
for image_path in new_image_paths:
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)[0]

    if prediction > 0.5:
        result = 'Poacher Detected'

    else:
        result = 'No Poacher Detected'
    print(f"Image: {image_path} - Prediction: {result}")

1/1 [==============================] - 1s 1s/step
Image: /content/drive/MyDrive/val/val1/img231.jpg - Prediction: No Poacher Detected
1/1 [==============================] - 0s 112ms/step
Image: /content/drive/MyDrive/val/val1/img228.jpg - Prediction: Poacher Detected
1/1 [==============================] - 0s 97ms/step
Image: /content/drive/MyDrive/val/val1/i2.jpg - Prediction: No Poacher Detected
1/1 [==============================] - 0s 97ms/step
Image: /content/drive/MyDrive/val/val1/img208.jpg - Prediction: Poacher Detected
1/1 [==============================] - 0s 85ms/step
Image: /content/drive/MyDrive/val/val1/img202.jpg - Prediction: Poacher Detected
1/1 [==============================] - 0s 57ms/step
Image: /content/drive/MyDrive/train/train1/056.jpg - Prediction: No Poacher Detected
